### Exploring Hacker News Posts
**Hacker News** is a site started by the startup incubator Y Combinator, where user-submitted stories (known as "posts") are voted and commented upon, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.
You can find the data set [here](https://www.kaggle.com/hacker-news/hacker-news-posts), but note that it has been reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions. 
Below are descriptions of the columns of the dataset:

 - `id`: The unique identifier from Hacker News for the post
 - `title`: The title of the post
 - `url`: The URL that the posts links to, if the post has a URL
 - `num_points`: The number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes
 - `num_comments`: The number of comments that were made on the post
 - `author`: The username of the person who submitted the post

 - `created_at`: The date and time at which the post was submitted

We're specifically interested in posts whose titles begin with either `Ask HN` or `Show HN`. Users submit `Ask HN` posts to ask the Hacker News community a specific question. Likewise, users submit `Show HN` posts to show the Hacker News community a project, product, or just generally something interesting.
 
We'll compare these two types of posts to determine the following:
- Do Ask HN or Show HN receive more comments on average?
- Do posts created at a certain time receive more comments on average?

In [1]:
from csv import reader
#open and read the file as a list of lists
opened_file = open("hacker_news.csv")
read_file = reader(opened_file)
hn_raw = list(read_file)

#separate and display the row containing the column header
header = hn_raw[0]
hn_raw = hn_raw[1:]
print("Headers of the table: ", header, "\n")
print("Total number of rows in dataset is: {}".format(len(hn_raw)))

# leave only rows with posts which had some comments
hn = []

for row in hn_raw:
    numb_comments = row[4]
    if numb_comments != "0":
        hn.append(row)

print("Number of commented posts in dataset is: {}".format(len(hn)))

# separate posts beginning with Ask HN and Show HN and append them to lists
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    if title.lower().startswith("ask hn"):
        ask_posts.append(row)
    elif title.lower().startswith("show hn"):
        show_posts.append(row)
    else:
        other_posts.append(row)
        
percentage_to_total_ask = round((len(ask_posts)/len(hn))*100, 2)
percentage_to_total_show = round((len(show_posts)/len(hn))*100, 2)

print("There are {} posts Ask HN".format(len(ask_posts)))
print("It is {}% of the total posts Ask HN with at least one comment".format(percentage_to_total_ask))
print("There are {} posts Show HN".format(len(show_posts)))
print("It is {}% of the total posts Show HN with at least one comment".format(percentage_to_total_show))
print("There are {} other posts".format(len(other_posts)))

#determine if ask posts or show posts receive more comments on average
ask_comments = 0
show_comments = 0
#calculate average number of comments for Ask HN
for row in ask_posts:
    num_comments = int(row[4])
    ask_comments += num_comments
avg_ask_comments = round(ask_comments/len(ask_posts))
#calculate average number of comments for Show HN
for row in show_posts:
    num_comments = int(row[4])
    show_comments += num_comments
avg_show_comments = round(show_comments/len(show_posts), 2)
#print the result
if avg_ask_comments > avg_show_comments:
    print("On average Ask HN posts get more comments as Show HN posts: {} versus {}".format(avg_ask_comments, avg_show_comments))
else:
    print("On average Show HN posts get more comments as Ask HN posts: {} versus {}".format(avg_show_comments, avg_ask_comments))

Headers of the table:  ['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'] 

Total number of rows in dataset is: 293119
Number of commented posts in dataset is: 80401
There are 6911 posts Ask HN
It is 8.6% of the total posts Ask HN with at least one comment
There are 5059 posts Show HN
It is 6.29% of the total posts Show HN with at least one comment
There are 68431 other posts
On average Ask HN posts get more comments as Show HN posts: 14 versus 9.81


As we can see, there are more posts Ask HN as Show HN as well as first ones attract more attention:
 - there are more posts Ask HN as Show HN: 8.6% versus 6.29% of the total number of posts with at least one comment
 - Ask HN posts get on average more comments as Show HN posts: 14% versus 9.81%

Further we will focus only *on posts Ask HN*

We'll determine if ask posts created at a *certain time* are more likely to attract comments. We'll use the following steps to perform this analysis:

- Calculate the amount of ask posts created in each hour of the day, along with the number of comments received.
- Calculate the average number of comments ask posts receive by hour created.


In [2]:
import datetime as dt

#count how many posts were created at any hour and how much comments did they get
counts_by_hour = {}
comments_by_hour = {}

for row in ask_posts:
    date_str = row[6]
    num_comments = int(row[4])
    #parse date from created_at column and extract hours
    date_1 = dt.datetime.strptime(date_str, "%m/%d/%Y %H:%M")
    hour = date_1.strftime("%H")
    
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
    else:
        counts_by_hour[hour] += 1
    if hour not in comments_by_hour:
        comments_by_hour[hour] = num_comments
    else:
        comments_by_hour[hour] += num_comments
        
#calculate average number of comments for any hour
avg_posts = []

for hour, num_comments in comments_by_hour.items():
    numb_posts = counts_by_hour[hour]
    avg_post = round(num_comments/numb_posts, 2)
    avg_posts.append([hour, avg_post])
    
#sort the list in reverse way and print line by line  
avg_posts.sort(key=lambda x:x[1], reverse = True)
for row in avg_posts:
    print(row)

['15', 39.67]
['13', 22.22]
['12', 15.45]
['10', 13.76]
['17', 13.73]
['02', 13.2]
['14', 13.15]
['04', 12.69]
['08', 12.43]
['22', 11.75]
['20', 11.38]
['11', 11.14]
['05', 11.14]
['21', 11.06]
['18', 10.79]
['16', 10.76]
['03', 10.16]
['07', 10.1]
['00', 9.86]
['19', 9.41]
['01', 9.37]
['06', 9.02]
['09', 8.39]
['23', 8.32]


## Conclusion
As our analysis has showed the most popular time to write a post on Hacker News is around 15:00, or following the dataset [documentation](https://www.kaggle.com/hacker-news/hacker-news-posts) 3:00 pm est. The longer time span to post - from 01 pm to 03 pm est.